In [1]:
pip install twofish

  Created wheel for twofish: filename=twofish-0.3.0-cp36-cp36m-linux_x86_64.whl size=27179 sha256=85fd79516dba1d9fe2c2d5ea28ed02d28d407529f952d6e89b8da8acca28d32b
  Stored in directory: /root/.cache/pip/wheels/e9/ce/00/66bd8b019c093719e2d78ea39a6f80c8133aecc1ec3f77f41c
Successfully built twofish


In [0]:
from twofish import Twofish
import pickle
import json
import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
from numpy import genfromtxt
import numpy as np
import pandas as pd
import base64
T = Twofish(b'*keythatcanbeanythig*')

In [0]:
def add_redun_bits(x):
  a = len(x)
  if a!=16:
    b=16-a
    for i in range (b):
      x=  x+' '
  return x

def remove(string, n):  
  first = string[:n]   
  last = string[n+1:]  
  return first + last

def remove_redun_bits(x):
  a = len(x)
  b = ''
  for i in range (a):
    if(x[i]!=' '):
      b = b+x[i]
  return b

def convert_to_binary(x):
  y = x.encode()
  return y

def encrypt(x):
  x = add_redun_bits(x)
  x = convert_to_binary(x)
  e = T.encrypt(x)
  return e

def decrypt(x):
  a = T.decrypt(x).decode()
  d = remove_redun_bits(a)
  return d


def remover_ext(filename):
  # print(type(filename))
  a = len(filename)
  name = ""
  for i in range (a):
    if (filename[i] == "."):
      return name
    else:
     name = name +filename[i]



def get_ext(filename):
  a = len(filename)
  ext = ""
  b = 0
  for i in range (a):
    if (filename[i] == "."):
      b = i
  ext = filename[b:a]
  return ext


def encrypt_file(filename):
  cat = get_ext(filename)
  if (cat==".png"):
    encrypt_img(filename)
  elif(cat == ".txt"):
    encrypt_txt(filename)
  elif(cat == ".csv"):
    encrypt_csv(filename)




def load_records():
  with open('out.pickle', 'rb') as f:
      b_dict = pickle.load(f)
  a= dict()
  for x, y in b_dict.items():
    x = decrypt(x)
    y = decrypt(y)
    a[x] =y
  return a



def load_hos_records():
  with open('hos_code.pickle', 'rb') as f:
      b_dict = pickle.load(f)
  a= dict()
  for x, y in b_dict.items():
    x = decrypt(x)
    y = decrypt(y)
    a[x] =y
  return a




def encrypt_csv(filename):
  name = remover_ext(filename)
  data = pd.read_csv(filename)
  b = len(data.columns)
  encpd = pd.DataFrame()
  for row in data.itertuples():
    enc = ['']*b
    for i in range (1,b+1):
      if(type(row[i])!=str):
        a = str(row[i])
      else:
        a = row[i]
      enc[i-1] = encrypt(a)
    ser = pd.Series(enc)
    encpd = encpd.append(ser,ignore_index=True)
  col = data.columns
  b = len(data.columns)
  for i in range (b):
    if(type(col[i])!=str):
      a = str(col[i])
    else:
      a = col[i]
    enc[i] = encrypt(a)
  ser = pd.Series(enc)
  encpd.columns = ser
  filename = "Enc_" + name + ".csv"
  encpd.to_csv((filename), index = False)
  filename = "Enc_" + name + ".pickle"
  encpd.to_pickle(filename)



def decrypt_csv(filename):
  name = remover_ext(filename)
  data = pd.read_pickle(filename)
  b = len(data.columns)
  encpd = pd.DataFrame()
  for row in data.itertuples():
    enc = ['']*b
    for i in range (1,b+1):
      a = row[i]
      enc[i-1] = decrypt(a)
    ser = pd.Series(enc)
    encpd = encpd.append(ser,ignore_index=True)
  col = data.columns
  b = len(data.columns)
  for i in range (b):
    a = col[i]
    enc[i] = decrypt(a)
  ser = pd.Series(enc)
  encpd.columns = ser
  filename = "Dec_" + name + ".csv"
  encpd.to_csv((filename), index = False)


def encrypt_txt(filename):
  f = open(filename, "r")
  name = remover_ext(filename)
  text = f.read()
  text_list = list()
  l = len(text)
  r = l%16
  temp = ""
  for i in range (l):
    if(text[i]!=' '):
      # temp = ['']*16
      l_no = int(i/16)
      if((len(temp)%16 ==0) and (l_no!=0)):
        text_list.append(temp)
        temp = ""
      temp = temp + text[i]
  text_list.append(temp)
  encrypted_text = list()
  for h in text_list:
    d = encrypt(h)
    encrypted_text.append(d)
  newfile = "Enc_" + (filename)
  with open(newfile, 'w') as filehandle:
    for listitem in encrypted_text:
        filehandle.write('%s\n' % listitem)
  newfile = "Enc_" + (name) + ".pickle"
  with open(newfile, 'wb') as f:
    pickle.dump(encrypted_text, f)


def decrypt_txt(filename):
  name = remover_ext(filename)
  with open(filename, 'rb') as f:
    encrypted_text = pickle.load(f)
  orgtxt = list()
  for h in encrypted_text:
    d = decrypt(h)
    orgtxt.append(d)
  newfile = "Dec_" + (name) + ".txt"
  with open((newfile), 'w') as filehandle:
    for listitem in orgtxt:
        filehandle.write('%s' % listitem)



def add_user(user,password):
  users = load_records()
  users[user] = password
  a= dict()
  # print(type(a))
  for x, y in users.items():
    x = encrypt(x)
    y = encrypt(y)
    # print(type(x))
    a[x] =y

  #print(type(a))
  with open('out.pickle', 'wb') as f:
      pickle.dump(a, f, protocol=pickle.HIGHEST_PROTOCOL)



def encrypt_img(filename):
  name = remover_ext(filename)
  input_file = open((filename),"rb")
  str = input_file.read()
  info = [str[i:i+16] for i in range(0, len(str), 16)]
  output = bytearray()
  a =0
  for i in info:
    if(len(i)==16):
      e = T.encrypt(i)
    else:
      ar = bytearray(i)
      while(len(ar)!=16):
        ar.append(ar[0])
        a+=1
      f = bytes(ar)
      temp = T.encrypt(f)
      temp = bytearray(temp)
      for i in range (a):
        temp = temp[:(-1)]
      e = bytes(temp)
    output+=(e)
  outbyte = bytes(output)

  # img = Image.frombytes("L", (100,100),outbyte) 
  # img = img.save('NAME.jpg')
  # img=Image.open('NAME.jpg').convert('L')
  # plt.imshow(img, cmap='gray')
  # plt.title("Encrypted image")
  # plt.show()
  
  newfile = "Enc_" + (name) + ".png"
  enc_file = open((newfile), "wb") 
  enc_file.write(outbyte)
  enc_file.close()




def decrypt_img(filename):
  name = remover_ext(filename)
  input_file = open((filename),"rb")
  str = input_file.read()
  info = [str[i:i+16] for i in range(0, len(str), 16)]
  output = bytearray()
  a =0
  for i in info:
    if(len(i)==16):
      e = T.decrypt(i)
    else:
      ar = bytearray(i)
      while(len(ar)!=16):
        ar.append(ar[0])
        a+=1
      f = bytes(ar)
      temp = T.decrypt(f)
      temp = bytearray(temp)
      for i in range (a):
        temp = temp[:(-1)]
      e = bytes(temp)
    output+=(e)
  outbyte = bytes(output)
  newfile = "Dec_" + (name) + ".png"
  enc_file = open((newfile), "wb") 
  enc_file.write(outbyte)
  enc_file.close()


In [0]:
# # save hospital's security code

# hospitals = {'abc': 'Abc@123', 'xyz': 'Xyz@123'}
# # a = my_dictionary() 
# a= dict()
# # print(type(a))
# for x, y in hospitals.items():
#   x = encrypt(x)
#   y = encrypt(y)
#   #print(type(x))
#   a[x] =y

# print(type(a))
# with open('hos_code.pickle', 'wb') as f:
#     pickle.dump(a, f, protocol=pickle.HIGHEST_PROTOCOL)

In [0]:
# # CODE TO GENERATE THE RECORD OF USERS AND STORE ENCYPTED FILE WITH RECORD.

# users = {'abc': 'Abc@123', 'xyz': 'Xyz@123'}
# # a = my_dictionary() 
# a= dict()
# # print(type(a))
# for x, y in users.items():
#   x = encrypt(x)
#   y = encrypt(y)
#   print(type(x))
#   a[x] =y

# print(type(a))
# with open('records.pickle', 'wb') as f:
#     pickle.dump(a, f, protocol=pickle.HIGHEST_PROTOCOL)
# #json.dump(a, open("text.txt",'w')) 